In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Adagrad
from tensorflow.keras.models import load_model
import struct
import numpy as np
import cv2

def __convert_to_one_hot(vector, num_classes):
    result = np.zeros(shape=[len(vector), num_classes])
    result[np.arange(len(vector)), vector] = 1
    return result

def __resize_image(src_image, dst_image_height, dst_image_width):
    src_image_height = src_image.shape[0]
    src_image_width = src_image.shape[1]

    if src_image_height > dst_image_height or src_image_width > dst_image_width:
        height_scale = dst_image_height / src_image_height
        width_scale = dst_image_width / src_image_width
        scale = min(height_scale, width_scale)
        img = cv2.resize(src=src_image, dsize=(0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    else:
        img = src_image

    img_height = img.shape[0]
    img_width = img.shape[1]
    dst_image = np.zeros(shape=[dst_image_height, dst_image_width], dtype=np.uint8)
    y_offset = (dst_image_height - img_height) // 2
    x_offset = (dst_image_width - img_width) // 2
    dst_image[y_offset:y_offset+img_height, x_offset:x_offset+img_width] = img
    return dst_image


def read_hoda_cdb(file_name):
    with open(file_name, 'rb') as binary_file:
        data = binary_file.read()
        offset = 0
        yy = struct.unpack_from('H', data, offset)[0]
        offset += 2
        m = struct.unpack_from('B', data, offset)[0]
        offset += 1
        d = struct.unpack_from('B', data, offset)[0]
        offset += 1
        H = struct.unpack_from('B', data, offset)[0]
        offset += 1
        W = struct.unpack_from('B', data, offset)[0]
        offset += 1
        TotalRec = struct.unpack_from('I', data, offset)[0]
        offset += 4
        LetterCount = struct.unpack_from('128I', data, offset)
        offset += 128 * 4
        imgType = struct.unpack_from('B', data, offset)[0]  # 0: binary, 1: gray
        offset += 1
        Comments = struct.unpack_from('256c', data, offset)
        offset += 256 * 1
        Reserved = struct.unpack_from('245c', data, offset)
        offset += 245 * 1

        if (W > 0) and (H > 0):
            normal = True
        else:
            normal = False

        images = []
        labels = []
        for i in range(TotalRec):
            StartByte = struct.unpack_from('B', data, offset)[0]
            offset += 1
            label = struct.unpack_from('B', data, offset)[0]
            offset += 1
            if not normal:
                W = struct.unpack_from('B', data, offset)[0]
                offset += 1
                H = struct.unpack_from('B', data, offset)[0]
                offset += 1
            ByteCount = struct.unpack_from('H', data, offset)[0]
            offset += 2
            image = np.zeros(shape=[H, W], dtype=np.uint8)
            if imgType == 0:
                for y in range(H):
                    bWhite = True
                    counter = 0
                    while counter < W:
                        WBcount = struct.unpack_from('B', data, offset)[0]
                        offset += 1
                        if bWhite:
                            image[y, counter:counter + WBcount] = 0  # Background
                        else:
                            image[y, counter:counter + WBcount] = 255  # ForeGround
                        bWhite = not bWhite  # black white black white ...
                        counter += WBcount
            else:
                data = struct.unpack_from('{}B'.format(W * H), data, offset)
                offset += W * H
                image = np.asarray(data, dtype=np.uint8).reshape([W, H]).T
            images.append(image)
            labels.append(label)
        return images, labels

def read_hoda_dataset(dataset_path, images_height=32, images_width=32, one_hot=False, reshape=True):
    images, labels = read_hoda_cdb(dataset_path)
    assert len(images) == len(labels)
    X = np.zeros(shape=[len(images), images_height, images_width], dtype=np.float32)
    Y = np.zeros(shape=[len(labels)], dtype=np.int)
    for i in range(len(images)):
        image = images[i]
        image = __resize_image(src_image=image, dst_image_height=images_height, dst_image_width=images_width)
        image = image / 255
        image = np.where(image >= 0.5, 1, 0)
        X[i] = image
        Y[i] = labels[i]

    if one_hot:
        Y = __convert_to_one_hot(Y, 10).astype(dtype=np.float32)
    else:
        Y = Y.astype(dtype=np.float32)

    if reshape:
        X = X.reshape(-1, images_height * images_width)
    else:
        X = X.reshape(-1, images_height, images_width, 1)

    return X, Y

In [2]:
img_rows = 100
img_cols = 100
num_classes = 10

X_train, Y_train = read_hoda_dataset(dataset_path='/content/drive/MyDrive/Train 60000.cdb',
                                images_height=img_rows,
                                images_width=img_cols,
                                one_hot=True,
                                reshape=True)

X_test, Y_test = read_hoda_dataset(dataset_path='/content/drive/MyDrive/Test 20000.cdb',
                              images_height=img_rows,
                              images_width=img_cols,
                              one_hot=True,
                              reshape=True)

X_remaining, Y_remaining = read_hoda_dataset('/content/drive/MyDrive/RemainingSamples.cdb',
                                             images_height=img_rows,
                                             images_width=img_cols,
                                             one_hot=True,
                                             reshape=True)

X_train = X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
X_test = X_test.reshape(X_test.shape[0],img_rows,img_cols,1)
X_remaining = X_remaining.reshape(X_remaining.shape[0],img_rows,img_cols,1)

<ipython-input-1-5070828769cc>:106: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y = np.zeros(shape=[len(labels)], dtype=np.int)


In [ ]:
# ADAM
def define_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform', input_shape=(img_rows, img_cols, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    opt = Adam(learning_rate=0.005, decay=0.8)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = define_model()

In [ ]:
print(model.summary())

batch_size = 256
epochs = 30

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
          
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("test_model.h5")
model = load_model('test_model.h5')

opt = Adam(learning_rate=0.005, decay=0.8)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 98, 98, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 batch_normalization_1 (Batc  (None, 46, 46, 128)      5

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
Y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(Y_test, axis=1)

conf_mat = confusion_matrix(Y_test, Y_pred)
print(conf_mat)

print('F1: ', f1_score(Y_test, Y_pred, average='weighted')*100)
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted')*100)
print('Precision: ', precision_score(Y_test, Y_pred, average='weighted')*100)
print('Accuracy: ', accuracy_score(Y_test, Y_pred)*100)

625/625 [==============================] - 9s 14ms/step
[[1995    1    0    0    1    1    1    0    0    1]
 [   2 1994    0    0    0    0    1    0    2    1]
 [   0   31 1924    0   26    0   12    0    0    7]
 [   3    1  127 1793   64    1    4    1    0    6]
 [   1    5    2    1 1981    0    7    0    0    3]
 [  25    0    0    0   18 1834    5    0  115    3]
 [   2    4    1    0    3    1 1960    0    1   28]
 [   7    5   16    0    1    0   54 1915    1    1]
 [   0    1    0    0    0    0    0    0 1997    2]
 [   2   10    0    0    0    0    5    0    8 1975]]
F1:  96.82442399199051
Recall:  96.84
Precision:  96.95026456157855
Accuracy:  96.84


In [13]:
# SGD
def define_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform', input_shape=(img_rows, img_cols, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())

    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    opt = SGD(momentum=0.9, learning_rate=0.04, decay=0.9)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = define_model()

In [14]:
print(model.summary())

batch_size = 256
epochs = 30

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
          
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("test_model.h5")
model = load_model('test_model.h5')

opt = SGD(momentum=0.9, learning_rate=0.04, decay=0.9)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 98, 98, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 batch_normalization_1 (Batc  (None, 46, 46, 128)      5

In [15]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
Y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(Y_test, axis=1)

conf_mat = confusion_matrix(Y_test, Y_pred)
print(conf_mat)

print('F1: ', f1_score(Y_test, Y_pred, average='weighted')*100)
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted')*100)
print('Precision: ', precision_score(Y_test, Y_pred, average='weighted')*100)
print('Accuracy: ', accuracy_score(Y_test, Y_pred)*100)

625/625 [==============================] - 9s 14ms/step
[[1983   12    0    0    0    2    0    0    2    1]
 [   5 1995    0    0    0    0    0    0    0    0]
 [   8  341 1623    3    0    0    1    8    0   16]
 [  18   47 1032  842    4    4    3   14    0   36]
 [  18   62  294  146 1224   54    5    2   25  170]
 [  96  105    0    0    3 1241    4    2  533   16]
 [  11  177   20    0    0    3 1585    4    6  194]
 [  15  181   26    0    0    0   13 1763    1    1]
 [   5   25    0    0    0    0    0    0 1968    2]
 [   6  240    0    0    0    0    3    0   12 1739]]
F1:  79.48922631630556
Recall:  79.815
Precision:  84.17374084621233
Accuracy:  79.815


In [3]:
# Adadelta
def define_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform', input_shape=(img_rows, img_cols, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())

    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    opt = Adadelta(learning_rate=0.01)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = define_model()

In [4]:
print(model.summary())

batch_size = 256
epochs = 30

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
          
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("test_model.h5")
model = load_model('test_model.h5')

opt = Adadelta(learning_rate=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 98, 98, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 batch_normalization_1 (Batc  (None, 46, 46, 128)      5

In [5]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
Y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(Y_test, axis=1)

conf_mat = confusion_matrix(Y_test, Y_pred)
print(conf_mat)

print('F1: ', f1_score(Y_test, Y_pred, average='weighted')*100)
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted')*100)
print('Precision: ', precision_score(Y_test, Y_pred, average='weighted')*100)
print('Accuracy: ', accuracy_score(Y_test, Y_pred)*100)

625/625 [==============================] - 9s 14ms/step
[[1999    0    0    0    0    0    0    1    0    0]
 [   0 1999    0    0    0    0    0    0    1    0]
 [   2   39 1957    0    0    0    0    0    0    2]
 [  13    0  109 1875    3    0    0    0    0    0]
 [   6   10   53   14 1907    0    3    0    0    7]
 [  86   30    2    0    4 1815    2    1   59    1]
 [   8   36    8    0    0    2 1914    4    2   26]
 [   2   16    6    0    0    0    0 1976    0    0]
 [   0   10    0    0    0    0    0    0 1990    0]
 [   8   42    0    0    0    0    1    0    4 1945]]
F1:  96.89120818878446
Recall:  96.885
Precision:  97.05313040464702
Accuracy:  96.885
